### Load data from parquet file, prepare the data and use Folium to create a map

In [ ]:
import folium
import calendar
from geopy.geocoders import Nominatim
import shapelets as sh
from shapelets.functions import h3Cell
from shapelets.apps import DataApp
from shapelets.apps.widgets import Text, Table

# Load data
playground = sh.sandbox()
oneMonthDS = playground.from_parquet('../../Benchmarks/nyc-taxi/2016/01/*.parquet')

# Find h3Cells
oneMonthDS = oneMonthDS.add_column("pickup", lambda row: h3Cell(row.pickup_longitude, row.pickup_latitude, 12))

# Run SQL query to find top tipping locations
hotspots = playground.execute_sql(f"""
    SELECT * FROM (
        SELECT
            t.pickup_latitude,
            t.pickup_longitude, 
            dayofweek(t.pickup_at), 
            avg(t.tip_amount), 
            row_number() OVER(
                PARTITION BY dayofweek(t.pickup_at) 
                ORDER BY avg(t.tip_amount) DESC
                ) AS ranking    
        FROM {oneMonthDS.alias} as t
        GROUP BY dayofweek(t.pickup_at), t.pickup_latitude, t.pickup_longitude
    ) 
    WHERE ranking <=3
    ORDER BY 3, 5
    LIMIT 1000
""").to_pandas()

# Create column with day names
hotspots['weekday'] = hotspots['dayofweek_t_pickup_at_'].apply(lambda x: calendar.day_name[x])

# Build folium map
m = folium.Map(location=[40.730610, -73.935242], zoom_start=10, tiles="Stamen Terrain")
colors = ['lightgreen', 'lightblue', 'pink', 'lightred', 'purple', 'lightgray', 'green']
for h in hotspots.iterrows(): 
    folium.Marker(
        location=[h[1].pickup_latitude, h[1].pickup_longitude],
        popup="Top #"+ str(int(h[1].ranking)) +"\ntip: $"+"{:.2f}".format(h[1]['avg_t_tip_amount_']) +
           "\nWeekday:"+calendar.day_name[int(h[1]['dayofweek_t_pickup_at_'])],
        icon=folium.Icon(icon='location-pin', color = colors[int(h[1]['dayofweek_t_pickup_at_'])])
    ).add_to(m)

### Simple data app (map only)

In [ ]:
# Create data app
app = DataApp("Top taxi tip locations", version=(1,0))

# Add map
map = app.folium_chart(folium=m)
app.place(map)

# Make data app available to group
app.register()

### Data app + full table

In [ ]:
# Create data app
app = DataApp("Top taxi tip locations", version=(2,0))

# Add map
map = app.folium_chart(folium=m)
app.place(map)

# Get addresses for top locations
hotspots['latlong'] = hotspots.pickup_latitude.astype(str) + ',' + hotspots.pickup_longitude.astype(str)
geolocator = Nominatim(user_agent="test")
hotspots['address'] = hotspots.latlong.apply(geolocator.reverse)

# Add table
table = app.table(data=hotspots[['weekday',
    'avg_t_tip_amount_',
    'ranking','address']].astype(str),
    rows_per_page=25)
app.place(table)

# Make data app available to group
app.register()

### Data app + selectors

In [ ]:
# Create data app
app = DataApp("Top taxi tip locations", version=(3,0))

# Add map
map = app.folium_chart(folium=m)
app.place(map)

# Get addresses for top locations
hotspots['latlong'] = hotspots.pickup_latitude.astype(str) + ',' + hotspots.pickup_longitude.astype(str)
geolocator = Nominatim(user_agent="test")
hotspots['address'] = hotspots.latlong.apply(geolocator.reverse)

# Define function
def getData(weekday, ranking)->Table:
    result = hotspots[(hotspots.weekday==weekday[1]) & (hotspots.ranking==ranking)] 
    return Table(data=result[['ranking','weekday','avg_t_tip_amount_','address']].astype(str))

# Add selectors
weekday_selector = app.selector(options = list(calendar.day_name))
app.place(weekday_selector)

ranking_selector = app.number_input(title="Select ranking",value=1,max=3)
app.place(ranking_selector)

# Add result string
result = app.table()
result.bind(getData,weekday_selector,ranking_selector,on_init=True)
app.place(result)

# Make data app available to group
app.register()

### Sharing the data app with user groups

In [ ]:
import shapelets as sh
sh.create_group("thegroup", "Example group")
sh.add_user_to_group("user1", groups=["thegroup"], write=False)
sh.add_user_to_group("user2", groups=["thegroup"], write=True)

### Deleting & listing groups

In [ ]:
sh.delete_group("thegroup")
sh.list_current_groups()